# 动态规划练习
递归的暴力解法 -> 带备忘录的递归解法 -> 非递归的动态规划解法

### 练习题 1
有一段楼梯有10级台阶，规定每一步只能跨一级或两级，要登上第10级台阶有几种不同的走法? (很像斐波那契数列)

In [3]:
# 递归的暴力解法
def solution_0(n):
    if (n == 1) or (n == 2):
        return 1
    else:
        return solution(n-1) + solution(n-2)

#### 分析：
时间复杂度： O(2^n)

问题：存在大量重复计算

In [23]:
# 带备忘录的递归解法--自顶向下--将重叠子问题结构存到备忘录
memory = [0 for i in range(11)]
def solution_1(n):
    if memory[n] != 0:
        return memory[n]
    if (n == 1) or (n == 2):
        memory[n] = 1
    else:
        memory[n] = solution_1(n-1) + solution_1(n-2)
    return memory[n]

#### 分析：
时间复杂度： O(n)

自顶向下的方法

In [21]:
# 非递归的动态规划--自底向上--关键在于状态转移方程
def solution_2(n):
    dp_table = [0 for i in range(n + 1)]
    dp_table[1] = dp_table[2] = 1
    for i in range(3, n+1):
        dp_table[i] = dp_table[i-1] + dp_table[i-2]
    return dp_table[n]

#### 分析：
时间复杂度： O(n)

关键在于找到状态转移方程，即暴力解形式

继续优化：当前状态只与前两个状态有关，降低空间复杂度

In [26]:
# 最佳解法
def solution_2(n):
    if (n == 1) or (n == 2):
        return 1
    pre = cur = 1
    for i in range(3, n+1):
        temp = pre + cur
        pre = cur
        cur = temp
    return cur

#### 分析：
时间复杂度：O(n)

空间复杂度：O(1)

就很舒服^_^

### 练习题 2 凑零钱
给 k 种面值的硬币，面值分别为 c1, c2 ... ck，再给一个总金额 n，问最少需要几枚硬币凑出这个金额，如果不可能凑出，则回答 -1 

例如 k = 3，面值分别为 1，2，5，总金额 n = 11，那么最少需要 3 枚硬币，即 11 = 5 + 5 + 1 

#### 分析
状态转移方程：
$
d(i) = 
\begin{cases}
0, & \text{if $n = 0$} \\
min(d(i-v_x)+1), & \text{if $n \neq 0$}
\end{cases}
$

In [30]:
# 暴力递归
def getCoinNum_0(coins, value):
    # 输入：硬币面值列表  总金额
    # 输出：最少金币数
    if value == 0:
        return 0
    num = 9999  # 需凑硬币数初始化为一个大数
    for coin in coins:
        if (value - coin) < 0:
            continue
        subNum = getCoinNum(coins, value - coin)  # 求解子问题
        if subNum == -1:
            continue
        num = min(num, subNum + 1)
    if num == 9999:
        return -1
    else:
        return num

In [45]:
# 带备忘录的递归
memory = [-2 for i in range(38+1)]
def getCoinNum_1(coins, value):
    # 输入：硬币面值列表  总金额
    # 输出：最少金币数
    num = 9999  # 需凑硬币数初始化为一个大数
    if memory[value] != -2:
        return memory[value]
    else:
        if value == 0:
            memory[value] = 0
        for coin in coins:
            if (value - coin) < 0:
                continue
            subNum = getCoinNum(coins, value - coin)  # 求解子问题
            if subNum == -1:
                continue
            num = min(num, subNum + 1)
        if num == 9999:
            memory[value] = -1
        else:
            memory[value] = num
    return memory[value]

In [78]:
# 非递归的动态规划
def getCoinNum_2(coins, value):
    # 输入：硬币面值列表  总金额
    # 输出：最少金币数
    dp_table = [9999 for i in range(value+1)]  # 初始化num一个大值
    dp_table[0] = 0
    for i in range(1, value+1):
        for coin in coins:
            if coin <= i:
                dp_table[i] = min(dp_table[i], dp_table[i - coin] +1)
    if dp_table[value] == 9999:
        return -1
    else:
        return dp_table[value]

In [79]:
getCoinNum_2([1, 3, 5], 11)

3

### 练习题 3 放苹果
把M个同样的苹果放在N个同样的盘子里，允许有的盘子空着不放，问共有多少种不同的分法？（用K表示）5，1，1和1，5，1 是同一种分法。

每个用例包含二个整数M和N。0<=m<=10，1<=n<=10。
#### 输入描述:
输入两个int整数
#### 输出描述:
输出结果，int型

#### 分析
设f(m,n)为m个苹果，n个盘子的放法数目，则先对n作讨论，

当n>m：则必定有n-m个盘子永远空着，去掉它们对摆放苹果方法数目不产生影响。即 if(n>m) f(m,n) = f(m,m)

当n <= m:不同的放法可以分成两类：含有0的方案数，不含有0的方案数

1. 含有0的方案数，即有至少一个盘子空着，即相当于 f(m,n)=f(m,n-1);

2. 不含有0的方案数，即所有的盘子都有苹果，相当于可以从每个盘子中拿掉一个苹果，不影响不同放法的数目，即 f(m,n)=f(m-n,n).而总的放苹果的放法数目等于两者的和，即 f(m,n)=f(m,n-1)+f(m-n,n)

递归出口条件说明：

* 当n=1时，所有苹果都必须放在一个盘子里，所以返回1；
* 当m==0(没有苹果可放)时，定义为1种放法；



In [1]:
# 暴力递归
def fun(m, n):
    if (m == 0) or (n == 1):
        return 1
    if m < n:
        return fun(m, m)
    else:
        return fun(m, n-1) + fun(m-n, n)
while True:
    try:
        M, N = map(int, input().split())
        print(fun(M, N))
    except:
        break

7 3
8



In [2]:
# 备忘录递归
def fun(m, n):
    if memory[m][n] == 0:
        if (m == 0) or (n == 1):
            memory[m][n] = 1
        elif m < n:
            memory[m][n] = fun(m, m)
        else:
            memory[m][n] = fun(m, n-1) + fun(m-n, n)
        return memory[m][n]
    else:
        return memory[m][n]
while True:
    try:
        M, N = map(int, input().split())
        memory = [[0 for j in range(N + 1)] for i in range(M + 1)]
        print(fun(M, N))
    except:
        break

7 3
8



In [3]:
# 非递归动态规划
def fun(m, n):
    dp_table = [[0 for j in range(n + 1)] for i in range(m + 1)]
    for i in range(n + 1):
        dp_table[0][i] = 1
    for i in range(m + 1):
        dp_table[i][1] = 1
    for j in range(1, n + 1):
        for i in range(1, m + 1):
            if i < j:
                dp_table[i][j] = dp_table[i][i]
            else:
                dp_table[i][j] = dp_table[i][j-1] + dp_table[i-j][j]
    return dp_table[m][n]
while True:
    try:
        M, N = map(int, input().split())
        print(fun(M, N))
    except:
        break

7 3
8



### 练习题 4 题目描述
请编写一个函数（允许增加子函数），计算n x m的棋盘格子（n为横向的格子数，m为竖向的格子数）沿着各自边缘线从左上角走到右下角，总共有多少种走法，要求不能走回头路，即：只能往右和往下走，不能往左和往上走。
#### 输入描述:
输入两个正整数
#### 输出描述:
返回结果

In [11]:
# 动态规划，状态转移方程和边界条件很容易找到
# 暴力递归
def fun(row, col):
    if (row == 0) or (col == 0):
        return 1
    return fun(row-1, col) + fun(row, col-1) 

while True:
    try:
        n, m = map(int, input().split())
        print(fun(n, m))
    except:
        break

2 2
6



In [1]:
# 带备忘录的动态规划写法
def fun(row, col):
    if (row == 0) or (col == 0):
        return memory[row][col]
    memory[row][col] = fun(row-1, col) + fun(row, col - 1)
    return memory[row][col]
while True:
    try:
        n, m = map(int, input().split())
        memory = [[1 for i in range(m+1)] for j in range(n+1)]
        print(fun(n, m))
    except:
        break

2 2
6



In [2]:
# 非递归动态写法
def fun(row, col):
    dp_table = [[1 for i in range(col+1)] for j in range(row+1)]
    for i in range(1, row+1):
        for j in range(1, col+1):
            dp_table[i][j] = dp_table[i-1][j] + dp_table[i][j-1]
    return dp_table[row][col]
while True:
    try:
        n, m = map(int, input().split())
        print(fun(n, m))
    except:
        break

2 2
6

